# Módulo de inferência econométrica - proc.py

Este arquivo reúne os testes de funções do módulo de processamento de dados do aplicativo que colete automaticamente dados da web e estime o modelo CAPM para ações na B3.

In [1]:
# Importação de bibliotecas

import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests

In [2]:
# Importar modulo es para escrever funções proc
import test_es

In [3]:
# Usando exemplo PETR4.SA

Ri = test_es.leitor_precos('VALE3.SA')
Rf = test_es.leitor_taxa()
Rm = test_es.leitor_indice()

In [4]:
Rf = np.array(Rf[-len(Ri):])
Rm = np.array(Rm[-len(Ri):])

# Cálculo de Prêmio de Risco

Premio = (Rf - Rm)

print(Premio.shape)

(253,)


## Requisito RU5 - método de estimação do modelo
Função estima_modelo()

Testes para a função de estimação do modelo por mínimos quadrados ordinários.

Ri= Rf + βi(Rm − Rf) + εi

In [5]:
def estima_modelo(Ri:np.array, Rm:np.array, Rf:np.array):
    pass

In [7]:
 # Construir DataFrame para análise
df = pd.DataFrame({'ativo': Ri, 'mercado': Rm, 'cdi': Rf})
df = df.dropna()

In [8]:
# Adicionando uma constante à variável independente
X = sm.add_constant(Premio)

print(X.shape)

(253, 2)


In [9]:
# Ajustar o modelo CAPM
results = sm.OLS(Ri, X, missing = 'drop').fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     25.16
Date:                Fri, 26 Jan 2024   Prob (F-statistic):           9.97e-07
Time:                        19:25:44   Log-Likelihood:                 428.47
No. Observations:                 253   AIC:                            -852.9
Df Residuals:                     251   BIC:                            -845.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0215      0.005      4.756      0.0

In [26]:
# Definindo a função de estimação por mínimos quadrados ordinários

def estima_modelo(Ri:np.array, Rm:np.array, Rf:np.array):
    Rf = np.array(Rf[-len(Ri):])
    Rm = np.array(Rm[-len(Ri):])
    Premio = (Rf - Rm)
    
    # Adicionando uma constante à variável independente
    X = sm.add_constant(Premio)
    
    # Estimando regressão
    results = sm.OLS(Ri, X, missing = 'drop').fit()
    return results

sigma = results.resid.std()
print(sigma)

## Requisito RU6 e RU7  - teste de nulidade do alfa de Jensen e do risco específico
Função testa_nulidade_t_parametro()

Testes para a função que realiza teste da nulidade do alfa de Jensen (o emprego do teste t).

O risco específico de cada ação deverá ser testado também com teste t.

In [ ]:
results.t_test('const = 0')

In [ ]:
# Teste de nulidade do Alfa de Jensen com teste t
test_nulidade_t = results.t_test('const = 0')

print(test_nulidade_t)

In [ ]:
# definindo função de teste de nulidade do Alfa de Jensen e do risco específico

def test_nulidade_t_parametro(results):
    teste_t = results.t_test('const = 0')
    return teste_t

## Requisito RU8 - teste do modelo
Função testa_nulidade_F_parametros()

Testes para a função que avalia a nulidade conjunta dos parâmetros da regressão (usando o teste F).

In [ ]:
#estabelecendo hipótese de nulidade

hipotese = "const = 0, x1 = 0"

print(results.f_test(hipotese))

In [ ]:
# Teste F para Nulidade Conjunta dos Coeficientes
test_nulidade_F_parametros = results.f_test("const = 0, x1 = 0")

print(test_nulidade_F_parametros)

In [ ]:
# definindo função de teste de nulidade do Alfa de Jensen e do risco específico

def test_nulidade_F_parametros(results):
    hipotese = "const = 0, x1 = 0"
    teste_f = results.f_test(hipotese)
    return teste_f

## Requisito RU9 - correlação serial 
Função testa_corr()

Testes para a função de teste da correlação serial do modelo (usando o correlograma).

In [ ]:
# bibliotecas
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Encontrando os resíduos da regressão
residuos = results.resid

print(residuos)

In [ ]:
# plotando correlograma

sm.graphics.tsa.plot_acf(residuos, lags=250)
plt.title("Correlograma dos Resíduos do CAPM")
plt.show()

In [ ]:
# definindo função de teste correlograma

def testa_corr(results):
    residuos = results.resid
    sm.graphics.tsa.plot_acf(residuos, lags=250)
    plt.title("Correlograma dos Resíduos do CAPM")
    plt.show()

### Modulo proc precisa retornar lista com:
results.params['Intercept']

results.params['Market']

results.resid.std()